In [1]:

import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DF2_Practice") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [79]:

iso_df = (spark.read
          .option("header","true")
          .option("inferSchema","true")
          .csv("sample.csv")
         )


In [80]:

from pyspark.sql.window import Window
from pyspark.sql.functions import lead
from pyspark.sql.functions import lag
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import when
from pyspark.sql.functions import *



In [81]:
window = Window.orderBy("dropoff_datetime").partitionBy("medallion")

In [82]:
iso_df = iso_df.withColumn("prev_pickup", lag("dropoff_datetime").over(window))
iso_df = iso_df.withColumn("delay", unix_timestamp("pickup_datetime") - unix_timestamp("prev_pickup"))
iso_df = iso_df.withColumn("session_ind", when(iso_df.delay >= 14400, 'N').otherwise('Y'))
mean_delay_df = iso_df.filter(iso_df.session_ind == 'Y').groupBy("medallion").agg(avg("delay").alias("mean_delay"))
iso_df = iso_df.join(mean_delay_df, on="medallion", how="left")

In [83]:
iso_df.head()

sel = iso_df.select("medallion", "pickup_datetime", "dropoff_datetime", "prev_pickup", "delay","session_ind",iso_df.mean_delay)
sel2 = iso_df.select( "Delay")


In [84]:
sel.show()

+--------------------+-------------------+-------------------+-------------------+-----+-----------+-----------------+
|           medallion|    pickup_datetime|   dropoff_datetime|        prev_pickup|delay|session_ind|       mean_delay|
+--------------------+-------------------+-------------------+-------------------+-----+-----------+-----------------+
|000318C2E3E638158...|2013-01-13 04:41:00|2013-01-13 04:55:00|               NULL| NULL|          Y|790.9090909090909|
|000318C2E3E638158...|2013-01-13 09:45:00|2013-01-13 09:47:00|2013-01-13 04:55:00|17400|          N|790.9090909090909|
|000318C2E3E638158...|2013-01-13 09:56:00|2013-01-13 10:04:00|2013-01-13 09:47:00|  540|          Y|790.9090909090909|
|000318C2E3E638158...|2013-01-13 10:05:00|2013-01-13 10:11:00|2013-01-13 10:04:00|   60|          Y|790.9090909090909|
|000318C2E3E638158...|2013-01-13 10:31:00|2013-01-13 10:41:00|2013-01-13 10:11:00| 1200|          Y|790.9090909090909|
|000318C2E3E638158...|2013-01-13 10:47:00|2013-0